# Imports

In [1]:
import math, pyperclip, os, re
from decimal import Decimal as _Decimal
import numpy as np
from functools import reduce
from itertools import product

# Docs

In [2]:
"""
Data types: Real, Complex, Int, Boolean, String, Undefined
Data structures: Scalar, Vector, Matrix, ...
Variables: v
Functions: fun
Named operators: v operation u
Conversion specifier: 45deg
Data properties: object.property
Oppertation: Numeric, Sets, Comparison 

Data Structures
Tensor: [a, b,, c, d,,, ...]
List: (a, b, c, ...)
Function Body: {expression}

Implicit Operations
2(expression)      *
(expression)2      *
var(expression)    *
fun(parameters)    call

Named operators
on two values: v operation u
on one value: v.operation

Input and Output conversion
1m + 12cm @ cm
2hr + 45min + 1hr + 30min @ datetime
0b1100 * 0xFF @ dec

Basic Operations:
Standard:       | Bitwise: int&bool | Comparison:           | 
+           add | ~             not | ==             equals | 
-      subtract | &&            and | !=          not equal | 
*      multiply | ||             or | <           less than | 
/        divide | <xor>         xor | >        greater then | 
//      int div | <<     left shift | <=      less or equal | 
%       modulus | >>    right shift | >=   greater or equal | 
^         power |                   |                       | 
!     factorial |                   |                       | 
|val|  absolute |                   |                       | 
=    assignment |                   |                       | 

Ternary Operators
= =                return values = function = body
a < x < b          between
a if cond else b   

Higher Ranking Data Structure Operations:
Vector:                 | Matrix:                          | Reductions:
<dot>       dot product | <matmul>   matrix multiplication | <all>
<cross>   cross product | |mat|                            | <any>
|vec|            length | .T              transpose matrix | <
.length          length | 
.angle            angle | 



1 + sqrt 4
1 + sqrt x
1 + $4
1 + 3 root 8
1 + b root x
1 + b $ x

10 nPr 2
n nPr r
10 nCr 2
n nCr r

[1, 2,, 3, 4] # [5, 6,, 7, 8]
[1, 2,, 3, 4].T
mat1.T

5*x

sin 30deg
sin pi
sin2 pi
sini 0.5

Function definitions
fun = x => x^2               single value output
fun = x => [x, x*2, x^2]     tensor outputs
fun = x => (x, x*2, x^2)     multiple outputs
fun = x => {                 piecewise function
    0,   if x < 0 ; 
    x^2, if 0 <= x <= 1 ; 
    x,   else
}

fun = (x,y,z) => {
    a = x+2 ; 
    b = y*2 ; 
    c = z^2 ; 
    (a,b,c)
}

if x < 0 do {
    x = 0
}

for i in 0:10 do {
    x += i
}

while x < 0 do {
    x += 1
}

@display = scientific 8
@display = hex


Statistical Operations:
"""
None

# Classes

## Token Matching

### TokenOperandDefinition

In [3]:
class TokenOperandDefinition():
    def __init__(self, token, precedence):
        
        self.token = token
        self.precedence = float(precedence)
        
    def __repr__(self):
        return str(self)
        
    def __str__(self):
        return f'TokenOperandDefinition({self.token}, {self.precedence})'

### TokenGroupDefinition

In [4]:
class TokenGroupDefinition():
    def __init__(self, open_token, close_token, function):
        
        self.open_token = open_token
        self.close_token = close_token
        
        self.function = function
        
    def __repr__(self):
        return str(self)
        
    def __str__(self):
        return f'TokenGroupDefinition({self.open_token}, {self.close_token})'

### TokenNewItemDefinition

In [5]:
class TokenNewItemDefinition():
    def __init__(self, token, depth):
        
        self.token = token
        self.depth = int(depth)
        
    def __repr__(self):
        return str(self)
        
    def __str__(self):
        return f'TokenNewItemDefinition({self.token}, {self.levels})'

## Token Tree Nodes

### Abstract NodeToken

In [6]:
class NodeToken():
    """
    represents a token that can be parsed
    """
    
    def is_complete(self):
        raise Exception('is_complete() not implemented')
        
    def set_left(self, node):
        raise Exception('set_left(node) not implemented')

    def set_right(self, node):
        raise Exception('set_right(node) not implemented')
        
    def get_right(self):
        raise Exception('get_right() not implemented')
        
    def get_evaluable(self):
        raise Exception('get_evaluable() not implemented')

### NodeBinary

In [7]:
class NodeBinary(NodeToken):
    """
    represents a token that can be parsed into a value/operation
    """
    def __init__(self, operation_definition, parent=None):
        self.parent = parent
        self.operation_definition = operation_definition
        self.precedence = operation_definition.precedence
        self.left = None
        self.right = None
    
    def is_complete(self):
        return self.left != None and self.right != None
        
    def set_left(self, node):
        self.left = node
        node.parent = self

    def set_right(self, node):
        self.right = node
        node.parent = self
        
    def get_right(self):
        return self.right
    
    def get_evaluable(self):
        return VariableFunction(self.operation_definition.token, (self.left.get_evaluable(), self.right.get_evaluable()))
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeBinary({self.operation_definition}, left={self.left}, right={self.right})'

### NodeUnaryLeft

In [8]:
class NodeUnaryLeft(NodeToken):
    
    def __init__(self, operation_definition, parent=None):
        self.parent = parent
        self.operation_definition = operation_definition
        self.precedence = operation_definition.precedence
        self.child = None
    
    def is_complete(self):
        return self.child != None
        
    def set_left(self, node):
        self.child = node
        node.parent = self

    def set_right(self, node):
        self.child = node
        node.parent = self
        
    def get_right(self):
        return self.child
    
    def get_evaluable(self):
        return VariableFunction(self.operation_definition.token, (self.child.get_evaluable(), ))
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeUnaryLeft({self.operation_definition}, child={self.child})'

### NodeUnaryRight

In [9]:
class NodeUnaryRight(NodeToken):
    
    def __init__(self, operation_definition, parent=None):
        self.parent = parent
        self.operation_definition = operation_definition
        self.precedence = operation_definition.precedence
        self.child = None
    
    def is_complete(self):
        return self.child != None
        
    def set_left(self, node):
        self.child = node
        node.parent = self

    def set_right(self, node):
        self.child = node
        node.parent = self
        
    def get_right(self):
        return self.child
    
    def get_evaluable(self):
        return VariableFunction(self.operation_definition.token, (self.child.get_evaluable(), ))
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeUnaryRight({self.operation_definition}, child={self.child})'

### NodeGroup

In [10]:
class NodeGroup(NodeToken):
    
    def __init__(self, group_definition, parent=None):
        self.parent = parent
        self.children = []
        
        self.group_definition = group_definition
        self.complete = False
        
        self.shape = {}
        self.sep_count = 0
    
    def is_complete(self):
        return self.complete

    def close(self):
        self.sep_count = 0
        self.complete = True
        
        expected_size = 1
        for d,s in self.shape.items(): expected_size *= s
        
        assert len(self.children) == expected_size, f'Inconsistent dimensions shape={self.shape}, expected={expected_size}, actual={len(self.children)}'
    
    def set_right(self, node):
        self.children[-1] = node
        node.parent = self
        
    def get_right(self):
        return self.children[-1]

    def add(self, node):
        self.children.append(node)
        node.parent = self
        
        if len(self.shape) > 0 and self.sep_count >= len(self.shape):
            for i in range(len(self.shape), self.sep_count+1):
                self.shape[i-1] = self.shape.get(i-1, 1)
            self.shape[self.sep_count-1] += 1
            
        self.sep_count = 0
        
    def increase(self, depth=1):
        self.sep_count += depth
        
        if len(self.shape) == 0:
            self.shape[0] = 1
        
        # ensure minumum number of dimensions exists
        for i in range(len(self.shape), self.sep_count+1):
            self.shape[i-1] = self.shape.get(i-1, 1)
    
    def get_shape(self):
        shape = list(self.shape.items())
        shape.sort(key=lambda x:x[0], reverse=True)
        shape = tuple(s for d,s in shape)
        return shape
    
    def get_children(self):
        return self.children
    
    def get_evaluable(self):
        
        evaluables = [v.get_evaluable() for v in self.children]
        
        return VariableTensor(evaluables, self.get_shape(), self.group_definition.function)
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeGroup({self.group_definition}, children={self.children})'

### NodeValue

In [11]:
class NodeValue(NodeToken):
    
    def __init__(self, value, value_type, parent=None):
        self.parent = parent
        self.value = value
        self.value_type = value_type
    
    def is_complete(self):
        return True
    
    def get_evaluable(self):

        value = None
        if self.value_type == TOKEN_TYPE_STRING:
            value = String(self.value)
            value = VariableTensor([value], ())
            
        if self.value_type == TOKEN_TYPE_INTEGER:
            value = Integer(self.value)
            value = VariableTensor([value], ())
            
        if self.value_type == TOKEN_TYPE_NUMBER:
            value = Real(self.value)
            value = VariableTensor([value], ())
            
        if self.value_type == TOKEN_TYPE_LITERAL:
            value = Variable(self.value)
        
        return value
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeValue({self.value}, {self.value_type})'

## Evaluable Tree Nodes

### Abstract Evaluable

In [12]:
class Evaluable():
    """
    represents a node that can be evaluated
    """
    
    def eval(self, environment):
        raise Exception('eval(environment) not implemented')

### Statement - results in one print

In [13]:
class Statement():
    
    def __init__(self, node):
        self.node = node
        
    def eval(self, environment):
        
        result = self.node.eval(environment)
        print(result)
        return result
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'Statement({self.node})'

### VariableFunction

In [14]:
class VariableFunction(Evaluable):
    
    def __init__(self, name, parameters):
        self.name = name
        self.parameters = parameters
        if isinstance(parameters, VariableTensor):
            self.parameters = parameters.data
    
    def eval(self, environment):
        
        assert self.name in environment, f'Function {self.name} not found'
        assert type(environment[self.name]) == FunctionSet, f'{self.name} is not a function'
        
        return environment[self.name].eval(environment, self.parameters)
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'VariableFunction_{self.name}({self.parameters})'

### Variable

In [15]:
class Variable(Evaluable):
    
    def __init__(self, name):
        self.name = name
    
    def eval(self, environment):
        
        if self.name not in environment:
            raise Exception(f'Variable {self.name} not found')
        
        return environment[self.name]
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f"Variable({self.name})"

### VariableTensor

In [16]:
class VariableTensor(Evaluable):
    
    def __init__(self, data, shape, function=None):
        self.data = data
        self.shape = shape
        self.function = function
    
    def eval(self, environment):
        
        values = [v.eval(environment) for v in self.data]
        
        if self.function == None and self.shape == ():
            return Tensor(values[:1], self.shape)
        else:
            return self.function(environment, values, self.shape)
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'VariableTensor({self.data}, {self.shape})'

## Data Structures

### DType [DEPRICATED]

In [17]:
# class DType():
    
#     def __init__(self, dtype=None, rank=None, shape=None):
#         self.dtype = dtype
#         self.rank = rank
#         self.shape = shape
        
#     def __repr__(self):
#         return str(self)
    
#     def __str__(self):
#         return f'DType({dtype}, {rank}, {shape})'
    
#     def __eq__(left, right):
#         return left.dtype == right.dtype and left.rank == right.rank and left.shape == right.shape
    
#     def __lt__(left, right):
        
#         if issubclass()
        
        
#     def __gt__(left, right):
#         return right < left
        
#     def __le__(left, right):
#         return left == right or left < right
        
#     def __ge__(left, right):
#         return left == right or left > right
        

### Tensor

In [18]:
class Tensor(Evaluable):
    """
    represents a value/tensor of any type, shape and rank
    """
    def __init__(self, data, shape=()):
        
        data = np.array(data, object)
        data = data.reshape((data.size,))
        
        assert data.size > 0, 'Tensor cannot be empty'
        
        if all(type(t) == Tensor for t in data):
            inner_shape = data[0].shape
            assert all(t.shape == inner_shape for t in data), 'Cannot combine tensors of different shapes'
            
            data = np.concatenate([t.data for t in data])
            shape = shape + inner_shape
        
        self._data = np.array(data, object).reshape(shape)
        self.data = self._data.reshape((self._data.size))
        
        self.first = self.data[0]
        self.shape = self._data.shape
        self.size = self._data.size
        self.rank = self._data.ndim
        self.dtype = type(self.data[0])
        
        assert all([type(i) == self.dtype for i in self.data]), 'All items in a Tensor must be of the same type'
        
    def __getitem__(self, index):
        
        data = self.data[index] if type(index) == int else self._data[index]
        shape = data.shape if type(data) == np.ndarray else ()
            
        return Tensor(data, shape)
        
    def __setitem__(self, index, value):
        
        if type(value) == Tensor:
            value = value._data
        
        if type(index) == int:
            self.data[index] = value
        else:
            self._data[index] = value
    
    def eval(self, environment):
        return self
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        
        if self.rank == 0:
            return f'Tensor({self.data[()]})'
        else:
            return f'Tensor({self.data}, {self.shape})'

### Array

In [19]:
class Array(Evaluable):
    
    def __init__(self, data):
        self.data = np.array(data, object)
        self.data = self.data.reshape((self.data.size,))
        self.size = self.data.size
        
    def __getitem__(self, index):
        return Array(self.data[index])
        
    def __setitem__(self, index, value):
        self.data[index] = value
    
    def eval(self, environment):
        return self
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'Array({self.data[()]})'

### FunctionSet

In [20]:
class FunctionSet(Evaluable):
    
    def __init__(self, name):
        self.name = name
        self.signatures = {}
        
    def __contains__(self, partial_signature):
        l = len(partial_signature)
        for signature in self.signatures:
            if signature[:l] == partial_signature:
                return True
            
        return False
    
    def __getitem__(self, signature):
        return self.signatures[signature]
    
    def __setitem__(self, signature, value):
        par_names, function = value
        self.signatures[signature] = (par_names, function)
        
    def add(self, function_signature):
        self.signatures[function_signature.parameter_types] = function_signature
        
    def eval(self, environment, parameters):
        
        parameters = list(parameters)
        signature = [type(p) for p in parameters]
        
        for i, parameter in enumerate(parameters):
            if tuple(signature[:i+1]) not in self:
                parameters[i] = parameter.eval(environment)
                signature[i] = parameters[i].dtype
        
        signature = tuple(signature)
        
        assert signature in self.signatures, f'Signature {self.name}{tuple(p.__name__ for p in signature)} has no matching overload'
            
        function_signature = self.signatures[signature]
        
        function = function_signature.function
        parameter_names = function_signature.parameter_names
        
        actual_shapes = [p.shape if type(p) == Tensor else () for p in parameters]
        actual_ranks = [len(s) for s in actual_shapes]
        
        ranks = [r if r!=None else a for r,a in zip(function_signature.parameter_ranks, actual_ranks)]
        extended_shapes = [s if r==0 else s[:-r] for s,r in zip(actual_shapes, ranks)]
        
        extended_shape = reduce(lambda x,y: x if len(x) > len(y) else y, extended_shapes)
        extended_ranks = [len(s) for s in extended_shapes]
        
        assert all([s in [extended_shape, ()] for s in extended_shapes]), f'Ranks, shape or extended shapes do not match'
        
        if extended_shape == ():
            return self.runFunction(function, environment, parameter_names, parameters)
        
        else:
            data = np.empty(extended_shape, dtype=object)
            
            for i in product(*[range(d) for d in extended_shape]):
                extracted_parameters = [p if r == 0 else p[i] for p, r in zip(parameters, extended_ranks)]
                
                data[i] = self.runFunction(function, environment, parameter_names, extracted_parameters)._data.tolist()
                    
            data = np.array(data.tolist())
            
            return Tensor(data, data.shape)
        
    def runFunction(self, function, environment, parameter_names, parameters):
        
        if parameter_names == None:
            return function(*parameters)

        else:
            new_scoped_environment = {**environment, **{k:v for k,v in zip(parameter_names, parameters)}}
            return function(new_scoped_environment)
        
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'FunctionSet({self.name}, {self.signatures})'

## Data Types

### Abstract Data

In [21]:
class Data(Evaluable):
    pass

### String

In [22]:
class String(Data):
    
    def __init__(self, value):
        self.value = value
    
    def eval(self, environment):
        return self

### Integer

In [23]:
class Integer(Data):
    
    def __init__(self, value):
        self.value = value
    
    def eval(self, environment):
        return self

### Real

In [24]:
class Real(Data):
    
    def __init__(self, value):
        self.value = float(value)
    
    def eval(self, environment):
        return self
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'Real({self.value})'

### Complex

### Boolean

In [25]:
class Boolean(Data):
    
    def __init__(self, value):
        self.value = value
    
    def eval(self, environment):
        return self

### FunctionSignature

In [26]:
class FunctionSignature():
    
    def __init__(self, name, function, parameter_names, parameter_types, parameter_ranks=None, parameter_shapes=None):
        
        self.name = name
        self.function = function
        self.parameter_count = len(parameter_types)
        self.parameter_names = parameter_names
        self.parameter_types = parameter_types
        self.parameter_ranks = parameter_ranks or self.parameter_count*(None,)
        self.parameter_shapes = parameter_shapes or self.parameter_count*(None,)
        
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'FunctionSignature{(self.name, self.function, self.parameter_names, self.parameter_types, self.parameter_ranks, self.parameter_shapes)}'

### Function [DEPTRICATED]

In [27]:
# class Function(Data):
        
#     def __init__(self, name):
#         self.name = name
#         self.signatures = {}
    
#     def eval(self, environment, parameters):
        
#         signature = tuple(p.dtype for p in parameters)
#         print('Function','eval', 'parameters', parameters, 'signature', signature)
            
#         function, parameter_names = None, []
        
#         for candidate_signature in self.signatures:
            
#             if len(signature) != len(candidate_signature):
#                 continue
            
#             if all(issubclass(a,b) for a,b in zip(signature, candidate_signature)):
#                 parameter_names, function = self.signatures[candidate_signature]
#                 break;
                
#         if function == None:
#             raise Exception(f'Signature {self.name}{tuple(p.__name__ for p in signature)} has no matching overload')
        
#         new_scoped_environment = {**environment, **{k:v for k,v in zip(parameter_names, parameters)}}
        
#         return function(new_scoped_environment)
    
#     def __getitem__(self, signature):
#         return self.signatures[signature]
    
#     def __setitem__(self, signature, value):
#         self.signatures[signature] = value
        
#     def __repr__(self):
#         return str(self)
    
#     def __str__(self):
#         return f'Function({self.name}, {self.signatures})'

## Built In Functions, Operators and Variables

### BuiltIns

In [28]:
class BuiltIns():
    
    def __init__(self):
        self.functions = []
        self.vars = []
        self.binary_operators = []
        self.left_unary_operators = []
        self.right_unary_operators = []
        self.groups = []
        self.new_items = []
        
    def register_function(self, name, function, parameter_names, parameter_types, parameter_ranks=None, parameter_shapes=None):
        self.functions.append((name, function, parameter_names, parameter_types, parameter_ranks, parameter_shapes))
        
    def register_binary_operator(self, name, function, precedence, parameter_names, parameter_types, parameter_ranks=None, parameter_shapes=None):
        self.register_function(name, function, parameter_names, parameter_types, parameter_ranks, parameter_shapes)
        self.binary_operators.append(TokenOperandDefinition(name, precedence))
        
    def register_left_unary_operator(self, name, function, precedence, parameter_names, parameter_types, parameter_ranks=None, parameter_shapes=None):
        self.register_function(name, function, parameter_names, parameter_types, parameter_ranks, parameter_shapes)
        self.left_unary_operators.append(TokenOperandDefinition(name, precedence))
        
    def register_right_unary_operator(self, name, function, precedence, parameter_names, parameter_types, parameter_ranks=None, parameter_shapes=None):
        self.register_function(name, function, parameter_names, parameter_types, parameter_ranks, parameter_shapes)
        self.right_unary_operators.append(TokenOperandDefinition(name, precedence))
        
    def register_var(self, name, value):
        self.vars.append((name, value))
        
    def register_grouping(self, open_token, close_token, function):
        self.groups.append(TokenGroupDefinition(open_token, close_token, function))
        
    def register_new_item_seperator(self, token, depth):
        self.new_items.append(TokenNewItemDefinition(token, depth))
        
        
built_ins = BuiltIns()

### Group ( )

In [29]:
def group_round(environment, items, shape):
    
    if shape == ():
        print(items)
        return items[0]
    else:
        return Array(items)

built_ins.register_grouping('(', ')', group_round)

### Group [ ]

In [30]:
def group_squre(environment, items, shape):
    
    assert len(items) > 0, 'Tensors can not be empty'
    
    base_shape = items[0].shape
    assert all([i.shape == base_shape for i in items]), 'Tensor elements have inconsistent shapes'
    
    extended_shape = shape + base_shape
    data = np.array([i.data for i in items]).reshape(extended_shape)
    
    return Tensor(data, extended_shape)

built_ins.register_grouping('[', ']', group_squre)

### Group { }

### Group | |

In [31]:
def group_straight(environment, items, shape):
    
    assert shape == (), 'Cannot take absolute value of an array'
    assert items[0].dtype == Real, 'Cannot take absolute value of non Real value'
    
#     return Tensor(Real(abs(items[0].first.value)))

    return VariableFunction('abs', items).eval(environment)

built_ins.register_grouping('|', '|', group_straight)
built_ins.register_grouping('||', '||', group_straight)

### New Item Seperators ',' and ';'

In [32]:
built_ins.register_new_item_seperator(',', 1)
built_ins.register_new_item_seperator(';', 2)

### Variabels

In [33]:
built_ins.register_var('PI', Tensor([Real(3.141592653589793238462643383279502884197)]))

### Binary Operantors: +, -, *, /, //, %, ^

In [34]:
# A + B
built_ins.register_binary_operator('+', lambda a, b: Tensor(Real(a.first.value + b.first.value)), 3, None, (Real, Real), (0, 0))

# A - B
built_ins.register_binary_operator('-', lambda a, b: Tensor(Real(a.first.value - b.first.value)), 3, None, (Real, Real), (0, 0))

# A * B
built_ins.register_binary_operator('*', lambda a, b: Tensor(Real(a.first.value * b.first.value)), 4, None, (Real, Real), (0, 0))

# A / B
built_ins.register_binary_operator('/', lambda a, b: Tensor(Real(a.first.value / b.first.value)), 4, None, (Real, Real), (0, 0))

# A // B
built_ins.register_binary_operator('//', lambda a, b: Tensor(Real(a.first.value // b.first.value)), 4, None, (Real, Real), (0, 0))

# A % B
built_ins.register_binary_operator('%', lambda a, b: Tensor(Real(a.first.value % b.first.value)), 4, None, (Real, Real), (0, 0))

# A ^ B
built_ins.register_binary_operator('^', lambda a, b: Tensor(Real(a.first.value ** b.first.value)), 5, None, (Real, Real), (0, 0))


### Matrix Multiplication: # #

In [35]:
def matmul(A, B):
    
    a_shape = A.shape
    b_shape = B.shape
    
    assert A.rank >= 2 and B.rank >= 2, 'left and right sides must be matrices'
    assert a_shape[-1] == b_shape[-2], 'left n_cols must equal right n_rows'
    
    n_rows = a_shape[0]
    n_cols = b_shape[1]
    n_vec = a_shape[1]
    shape = (n_rows , n_cols)
    
    data = np.empty(shape, dtype=object)
    
    for row in range(n_rows):
        for col in range(n_cols):
            v = 0
            for i in range(n_vec):
                v += A[(row, i)].first.value * B[(i, col)].first.value
            data[row,col] = Real(v)
    
    return Tensor(data, shape)

built_ins.register_binary_operator('#', matmul, 4, None, (Real, Real), (2, 2))


### Absolute value abs()

In [50]:
def absolute(A):
    return Tensor(Real(abs(A.first.value)))

built_ins.register_left_unary_operator('abs', absolute, 2, None, (Real,), (0,))

# Define Token

## Tokens Types

In [37]:
    
# operand kinds
# group, close_group, return_to_group, binary, unary_left, unary_right, *_ = enum(10)
# (
#     OPERAND_TYPE_GROUP, 
#     OPERAND_TYPE_CLOSE_GROUP, 
#     OPERAND_TYPE_RETURN_TO_GROUP, 
#     OPERAND_TYPE_BINARY, 
#     OPERAND_TYPE_UNARY_LEFT, 
#     OPERAND_TYPE_UNARY_RIGHT, 
# *_) = enum(10)


TOKEN_TYPE_STRING =   'string'
TOKEN_TYPE_INTEGER =  'integer'
TOKEN_TYPE_NUMBER =   'number'
TOKEN_TYPE_OPERATOR = 'operand'
TOKEN_TYPE_OPEN_GROUP =    'group'
TOKEN_TYPE_LITERAL =  'literal'

TOKEN_TYPE_OPEN_GROUP = 'open group'
TOKEN_TYPE_CLOSE_GROUP = 'close group'
TOKEN_TYPE_NEW_ITEM    = 'new item'
TOKEN_TYPE_BINARY      = 'binary operand'
TOKEN_TYPE_UNARY_LEFT  = 'left unary operand'
TOKEN_TYPE_UNARY_RIGHT = 'right unary operand'

OPERAND_TYPES = [
    TOKEN_TYPE_OPEN_GROUP,
    TOKEN_TYPE_CLOSE_GROUP, 
    TOKEN_TYPE_NEW_ITEM, 
    TOKEN_TYPE_BINARY, 
    TOKEN_TYPE_UNARY_LEFT, 
    TOKEN_TYPE_UNARY_RIGHT, 
]

TOKEN_TYPE_VALUE = [
    TOKEN_TYPE_STRING,
    TOKEN_TYPE_INTEGER,
    TOKEN_TYPE_NUMBER,
    TOKEN_TYPE_LITERAL,
]

## Token Operations

In [38]:
# binary_operands = [
#     Operand(r'.',   8), 
#     Operand(r':',   6), 
#     Operand(r'^',   5), 
#     Operand(r'*',   4), 
#     Operand(r'/',   4), 
#     Operand(r'//',  4), 
#     Operand(r'%',   4), 
#     Operand(r'+',   3), 
#     Operand(r'-',   3), 
#     Operand(r'&&',  2), 
#     Operand(r'||',  2), 
#     Operand(r'xor', 2), 
#     Operand(r'==',  1), 
#     Operand(r'!=',  1), 
#     Operand(r'>',   1), 
#     Operand(r'>=',  1), 
#     Operand(r'<',   1), 
#     Operand(r'<=',  1), 
#     Operand(r'=',   0),
# ]

# binary_operands = built_ins.binary_operators

# left_unary_operands = [
#     Operand(r'+', 7), 
#     Operand(r'-', 7), 
#     Operand(r'~', 7), 
#     Operand(r'=', 0),
# ]

# right_unary_operands = [
#     Operand(r'!', 7),
# ]

# new_item = [
#     TokenNewItemDefinition(',', 1),
#     TokenNewItemDefinition(';', 2),
# ]

# groups = [
#     Group(r'\(', r')'),
#     Group(r'\[', r']'),
#     Group(r'\{', r'}'),
#     Group(r'[a-zA-Z][a-zA-Z0-9_]*\(', r')'),
# ]

# close_group = [Operand(g.close_op, -1) for g in groups]

# sort by length
# binary_operands.sort(key = lambda o:(len(o.match), o.precedence), reverse=True)
# left_unary_operands.sort(key = lambda o:(len(o.match), o.precedence), reverse=True)
# right_unary_operands.sort(key = lambda o:(len(o.match), o.precedence), reverse=True)
# new_item.sort(key = lambda o:(len(o.match), o.precedence), reverse=True)

# (
#     binary_operands,
#     left_unary_operands,
#     right_unary_operands,
#     new_item,
#     close_group,
# )

# TODO sort built in tokens

token_definitions = {
    TOKEN_TYPE_OPEN_GROUP:  built_ins.groups,
    TOKEN_TYPE_CLOSE_GROUP: built_ins.groups, 
    TOKEN_TYPE_NEW_ITEM:    built_ins.new_items, 
    TOKEN_TYPE_BINARY:      built_ins.binary_operators, 
    TOKEN_TYPE_UNARY_LEFT:  built_ins.left_unary_operators, 
    TOKEN_TYPE_UNARY_RIGHT: built_ins.right_unary_operators, 
}

## Operation Indexing

In [39]:
def find_token_definition(string, token_type, f = lambda x:x.token):
    for token_definition in token_definitions[token_type]:
        if string == f(token_definition):
#         if re.fullmatch(operator.re_match, string) != None:
            return token_definition
    return None

## Define Token Regular Expressions

In [40]:
def re_join(l, f=lambda x:x):
    items = [f(i) for i in l]
    items.sort(key=lambda x:len(x), reverse=True)
    return '(' + ')|('.join([re.escape(i) for i in items]) + ')'

# regex
re_number =    r"""((([\.][0-9]+)|([0-9]+[\.]?[0-9]*))([eE][-+]?[0-9]+)?[a-z]*)"""
re_integer =   r"""((0b|0o|0d|0x|[0-9]+_)[0-9a-zA-Z,]+)"""
re_string =    r"""((\""".*?\""")|('''.*?''')|(".*?")|('.*?'))"""
re_literal =   r"""([A-Za-z_][A-Za-z0-9_]*)"""


re_open_group =           re_join(built_ins.groups, lambda x:x.open_token)
re_close_group =          re_join(built_ins.groups, lambda x:x.close_token)
re_binary_operands =      re_join(built_ins.binary_operators, lambda x:x.token)
re_left_unary_operands =  re_join(built_ins.left_unary_operators, lambda x:x.token)
re_right_unary_operands = re_join(built_ins.right_unary_operators, lambda x:x.token)
re_new_item =             re_join(built_ins.new_items, lambda x:x.token)


re_tokens = {
    TOKEN_TYPE_STRING:      re_string,
    TOKEN_TYPE_INTEGER:     re_integer,
    TOKEN_TYPE_NUMBER:      re_number,
    TOKEN_TYPE_LITERAL:     re_literal,
    TOKEN_TYPE_OPEN_GROUP:  re_open_group,
    TOKEN_TYPE_BINARY:      re_binary_operands,
    TOKEN_TYPE_UNARY_LEFT:  re_left_unary_operands,
    TOKEN_TYPE_UNARY_RIGHT: re_right_unary_operands,
    TOKEN_TYPE_NEW_ITEM:    re_new_item,
    TOKEN_TYPE_CLOSE_GROUP: re_close_group,
}

re_tokens

{'string': '((\\""".*?\\""")|(\'\'\'.*?\'\'\')|(".*?")|(\'.*?\'))',
 'integer': '((0b|0o|0d|0x|[0-9]+_)[0-9a-zA-Z,]+)',
 'number': '((([\\.][0-9]+)|([0-9]+[\\.]?[0-9]*))([eE][-+]?[0-9]+)?[a-z]*)',
 'literal': '([A-Za-z_][A-Za-z0-9_]*)',
 'open group': '(\\|\\|)|(\\()|(\\[)|(\\|)',
 'binary operand': '(\\/\\/)|(\\+)|(\\-)|(\\*)|(\\/)|(\\%)|(\\^)|(\\#)',
 'left unary operand': '()',
 'right unary operand': '()',
 'new item': '(\\,)|(\\;)',
 'close group': '(\\|\\|)|(\\))|(\\])|(\\|)'}

# Parse

## Lexing

In [41]:
def re_match_length(string, re_pattern):
    match = re.match(re_pattern, string)
    return match.span()[1] if match != None else 0

# TOKEN_TYPE_STRING
# TOKEN_TYPE_INTEGER
# TOKEN_TYPE_NUMBER
# TOKEN_TYPE_LITERAL

# TOKEN_TYPE_OPEN_GROUP
# TOKEN_TYPE_BINARY
# TOKEN_TYPE_UNARY_LEFT
# TOKEN_TYPE_UNARY_RIGHT
# TOKEN_TYPE_NEW_ITEM
# TOKEN_TYPE_CLOSE_GROUP

def lexing(string, ans_available=False):
    tokens = []

    i = 0
    while i < len(string):
            
        last_token_type = tokens[-1][0] if len(tokens) > 0 else None
        allowed_token_types = []

        # begin with
        if last_token_type in [
            None,
        ]:
            
            allowed_token_types = [TOKEN_TYPE_BINARY] if ans_available else []
            allowed_token_types += [
                TOKEN_TYPE_UNARY_LEFT,
                TOKEN_TYPE_OPEN_GROUP,
                TOKEN_TYPE_NEW_ITEM,
                TOKEN_TYPE_CLOSE_GROUP,
                *TOKEN_TYPE_VALUE,
            ]
        
        if last_token_type in [\
            TOKEN_TYPE_OPEN_GROUP,
            TOKEN_TYPE_NEW_ITEM,
        ]:
            allowed_token_types = [
                TOKEN_TYPE_UNARY_LEFT,
                TOKEN_TYPE_OPEN_GROUP,
                TOKEN_TYPE_NEW_ITEM,
                TOKEN_TYPE_CLOSE_GROUP,
                *TOKEN_TYPE_VALUE,
            ]

        # after value
        if last_token_type in [
            TOKEN_TYPE_CLOSE_GROUP,
            TOKEN_TYPE_UNARY_RIGHT,
            *TOKEN_TYPE_VALUE,
        ]:
            allowed_token_types = [
                TOKEN_TYPE_UNARY_RIGHT,
                TOKEN_TYPE_BINARY,
                TOKEN_TYPE_NEW_ITEM,
                TOKEN_TYPE_CLOSE_GROUP,
            ]

        # after operator
        if last_token_type in [
            TOKEN_TYPE_BINARY,
            TOKEN_TYPE_UNARY_LEFT,

        ]:
            allowed_token_types = [
                TOKEN_TYPE_OPEN_GROUP,
                TOKEN_TYPE_UNARY_LEFT,
                *TOKEN_TYPE_VALUE,
            ]
            
        
        # find matching token type
        token_type, token_str = None, None
        
        for possible_token_type in allowed_token_types:
            re_pattern = re_tokens[possible_token_type]
            
            l = re_match_length(string[i:], re_pattern)
            if l > 0:
                token_type = possible_token_type
                token_str = string[i:i+l]
                break
                

                    
        # invalid token
        if token_type == None:
#             raise Exception(f"Token not allowed at position {i}")
            i += 1
        else:
            tokens.append((token_type, token_str))
            i += len(token_str)

    return tokens


## Treeify

In [42]:
def bubble_up(focus, new):
    """
    Finds ancestor/parent in tree upwards from `token` thats the first group token or the first
    """
    
    while True:
#         print('bubble_up', focus.value, type(focus))
#         if type(focus) != NodeValue and (type(focus) == NodeGroup or focus.precedence < new.precedence):
        if type(focus) == NodeGroup or focus.precedence < new.precedence:
#             print('return', focus.value)
            return focus
        else:
            focus = focus.parent
        
def bubble_up_to_group(focus):
    while True:
        if type(focus) == NodeGroup:
            return focus
        else:
            focus = focus.parent

# TOKEN_TYPE_STRING
# TOKEN_TYPE_INTEGER
# TOKEN_TYPE_NUMBER
# TOKEN_TYPE_LITERAL

# TOKEN_TYPE_OPEN_GROUP
# TOKEN_TYPE_BINARY
# TOKEN_TYPE_UNARY_LEFT
# TOKEN_TYPE_UNARY_RIGHT

# TOKEN_TYPE_NEW_ITEM
# TOKEN_TYPE_CLOSE_GROUP

def build_token_tree(tokens):
    
    root = NodeGroup("ROOT", None)
    focus = root
    
    for token_type, value in tokens:
        
        
        # focus is
        # Binary Operator
        # Unary Operator
        if type(focus) in [
            NodeBinary,
            NodeUnaryLeft,
        ]:
            
            # next is 
            # Unary left
            if token_type == TOKEN_TYPE_UNARY_LEFT:
#                 print('insert lef unary')
                
                operand = find_token_definition(value, TOKEN_TYPE_UNARY_LEFT)
                next_node = NodeUnaryLeft(operand)
                
                focus.set_right(next_node)
                focus = next_node
                
            # next is
            # Group
            elif token_type == TOKEN_TYPE_OPEN_GROUP:
#                 print('insert open group')
                
                group = find_token_definition(value, TOKEN_TYPE_OPEN_GROUP, lambda x:x.open_token)
                next_node = NodeGroup(group)
                focus.set_right(next_node)
                focus = next_node
            
            # next is
            # TOKEN_TYPE_STRING
            # TOKEN_TYPE_INTEGER
            # TOKEN_TYPE_NUMBER
            # TOKEN_TYPE_LITERAL
            elif token_type in TOKEN_TYPE_VALUE:
#                 print('insert value')
                
                next_node = NodeValue(value, token_type)
                focus.set_right(next_node)
                focus = next_node
                
            else:
                raise Exception(f"token '{value}' not allowed here")
            
            
        # focus is
        # Value
        # Closed Group
        elif type(focus) == NodeValue or (type(focus) == NodeGroup and focus.is_complete()):
            
            # next is
            # Binary
            if token_type == TOKEN_TYPE_BINARY:
#                 print('insert binary')
                
                operand = find_token_definition(value, TOKEN_TYPE_BINARY)
                next_node = NodeBinary(operand)
                    
                parent_node = bubble_up(focus.parent, next_node)
                child_node = parent_node.get_right()
                parent_node.set_right(next_node)
                next_node.set_left(child_node)
                focus = next_node
                
            
            # next is
            # Unary right
            elif token_type == TOKEN_TYPE_UNARY_RIGHT:
#                 print('insert right unary')
                
                operand = find_token_definition(value, TOKEN_TYPE_UNARY_RIGHT)
                next_node = NodeUnaryRight(operand)
                    
                parent_node = focus.parent # bubble_up(focus.parent, next_node)
                child_node = parent_node.get_right()
                parent_node.set_right(next_node)
                next_node.set_left(child_node)
                # focus = next_node
            
            # next is
            # New item
            elif token_type == TOKEN_TYPE_NEW_ITEM:
#                 print('new item')
                
                token_definition = find_token_definition(value, TOKEN_TYPE_NEW_ITEM)
    
                parent_node = bubble_up_to_group(focus.parent)
                parent_node.increase(token_definition.depth)
                focus = parent_node
            
            # next is
            # New item
            elif token_type == TOKEN_TYPE_CLOSE_GROUP:
#                 print('close group')
                
                parent_node = bubble_up_to_group(focus.parent)
                parent_node.close()
                focus = parent_node
            
            # next is
            else:
                raise Exception(f"token '{value}' not allowed here")
#                 print('start new item in parent group')
        
        
        # focus is
        # Open Group
        elif type(focus) == NodeGroup and not focus.is_complete(): 
            
            # next is
            # Binary - use ans on left
            if token_type == TOKEN_TYPE_BINARY:
#                 print('insert binary with ans as left')

                left = NodeValue('ans', TOKEN_TYPE_LITERAL)
                
                operand = find_token_definition(value, TOKEN_TYPE_BINARY)
                next_node = NodeBinary(operand)
                next_node.set_left(left)
                
                focus.add(next_node)
                focus = next_node
            
            # next is
            # Unary left
            elif token_type == TOKEN_TYPE_UNARY_LEFT:
#                 print('add unary left')
                
                operand = find_token_definition(value, TOKEN_TYPE_UNARY_LEFT)
                next_node = NodeUnaryLeft(operand)
                
                focus.add(next_node)
                focus = next_node
            
            # next is
            # Group
            elif token_type == TOKEN_TYPE_OPEN_GROUP:
#                 print('add open group')
                
                group = find_token_definition(value, TOKEN_TYPE_OPEN_GROUP, lambda x:x.open_token)
                next_node = NodeGroup(group)
                focus.add(next_node)
                focus = next_node
            
            # next is
            # New item
            elif token_type == TOKEN_TYPE_CLOSE_GROUP:
#                 print('close group')
                
                focus.close()
            
            # next is
            # TOKEN_TYPE_STRING
            # TOKEN_TYPE_INTEGER
            # TOKEN_TYPE_NUMBER
            # TOKEN_TYPE_LITERAL
            elif token_type in TOKEN_TYPE_VALUE:
#                 print('add value')
                
                next_node = NodeValue(value, token_type)
                focus.add(next_node)
                focus = next_node
                
            # next is 
            # TOKEN_TYPE_NEW_ITEM
            elif token_type == TOKEN_TYPE_NEW_ITEM:
#                 print('new item/dimension')
                
                focus.increase()
                
            # next is
            else:
                raise Exception(f"token '{value}' not allowed here")
#                 print('start new item in parent group')
            
        
    
    return root
    

## Computation Graph

In [43]:
def build_computation_graphs(token_tree):
    
    satatement_root_tokens = token_tree.get_children()
    statements = [Statement(t.get_evaluable()) for t in satatement_root_tokens]
    
    return statements

# Excecute

## Evaluate

In [44]:
def evaluate(computation_graphs, environment):
    return [g.eval(environment) for g in computation_graphs]

## Run

In [45]:
def calc(query, ans_available=False, debug=False):
        
#     commands
#     if   query == 'exit':   break
#     elif query == 'help':   help()
#     elif query == 'ref':    ref()
#     elif query == 'clear':  clear()
#     elif query == 'copy':   pyperclip.copy(ans)
#     elif query == '=':      pyperclip.copy(ans)

#     # evaluate query
#     elif query != "":
    
    built_in_variables = {k:v for k,v in built_ins.vars}
    built_in_functions = {}
    
    for name, function, parameter_names, parameter_types, parameter_ranks, parameter_shapes in built_ins.functions:
        
        built_in_functions[name] = built_in_functions.get(name, FunctionSet(name))
        built_in_functions[name].add(FunctionSignature(
            name, 
            function, 
            parameter_names, 
            parameter_types, 
            parameter_ranks, 
            parameter_shapes
        ))
        
    
    environment = {**built_in_variables, **built_in_functions}
    if debug: print(environment, end='\n\n')
    
    tokens = lexing(query, ans_available)
    if debug: print(tokens, end='\n\n')
    
    token_tree = build_token_tree(tokens)
    if debug: print(token_tree, end='\n\n')
    
    computation_graphs = build_computation_graphs(token_tree)
    if debug: print(computation_graphs, end='\n\n')
    
    results = evaluate(computation_graphs, environment)

    return results

## Test

In [46]:
calc('(1 + [1,2,3]) * 2, 1 + [1,2,3] * 2')

[Tensor([Real(2.0) Real(3.0) Real(4.0)], (3,))]
Tensor([Real(4.0) Real(6.0) Real(8.0)], (3,))
Tensor([Real(3.0) Real(5.0) Real(7.0)], (3,))


[Tensor([Real(4.0) Real(6.0) Real(8.0)], (3,)),
 Tensor([Real(3.0) Real(5.0) Real(7.0)], (3,))]

In [47]:
calc('2 ^ [1,2,3,4], [1,2,3,4] ^ 2, [2,4,8] ^ [2,3,4], PI * [1/2, 1, 2,,,]')

Tensor([Real(2.0) Real(4.0) Real(8.0) Real(16.0)], (4,))
Tensor([Real(1.0) Real(4.0) Real(9.0) Real(16.0)], (4,))
Tensor([Real(4.0) Real(64.0) Real(4096.0)], (3,))
Tensor([Real(1.5707963267948966) Real(3.141592653589793) Real(6.283185307179586)], (1, 1, 3))


[Tensor([Real(2.0) Real(4.0) Real(8.0) Real(16.0)], (4,)),
 Tensor([Real(1.0) Real(4.0) Real(9.0) Real(16.0)], (4,)),
 Tensor([Real(4.0) Real(64.0) Real(4096.0)], (3,)),
 Tensor([Real(1.5707963267948966) Real(3.141592653589793) Real(6.283185307179586)], (1, 1, 3))]

In [48]:
calc('[1,2,,3,4,,5,6] # [1,2,3,,4,5,6]')

Tensor([Real(9.0) Real(12.0) Real(15.0) Real(19.0) Real(26.0) Real(33.0)
 Real(29.0) Real(40.0) Real(51.0)], (3, 3))


[Tensor([Real(9.0) Real(12.0) Real(15.0) Real(19.0) Real(26.0) Real(33.0)
  Real(29.0) Real(40.0) Real(51.0)], (3, 3))]

In [56]:
calc('|0 - |10-100| |, |0-1|, ||0-1||, |[1,0-1,2,0-2]|')

Tensor([Real(90.0)])
Tensor([Real(1.0)])
Tensor([Real(1.0)])
Tensor([Real(1.0) Real(1.0) Real(2.0) Real(2.0)], (4,))


[Tensor([Real(90.0)]),
 Tensor([Real(1.0)]),
 Tensor([Real(1.0)]),
 Tensor([Real(1.0) Real(1.0) Real(2.0) Real(2.0)], (4,))]